## **1. Import các thư viện cần thiết**

In [14]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import re
import nltk
import unidecode

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **2. Tiền xử lý**

In [15]:
english_stop_words = stopwords.words('english') # Lấy danh sách stopwords từ thư viện ntlk
stemmer = PorterStemmer() # Khai báo stemmer object (dùng để stemming trong hàm normalize text)

# Xây dựng hàm text normalization
def text_normalize(text):
    text = text.lower() # Chuyển chữ viết thường 
    text = unidecode.unidecode(text) # Mã hóa về ASCII
    text = text.strip() # Xóa kí tự đặc biệt ở đầu và cuối string
    text = re.sub(r'[^\w\s]', '', text) # Loại bỏ dấu câu
    text = ' '.join([word for word in text.split(' ') if word not in english_stop_words]) # Xóa stopwords
    text = ' '.join([stemmer.stem(word) for word in text.split(' ')]) # Stemming
 
    return text

In [16]:
BATCH_SIZE = 128
MAX_SEQ_LEN = 128
MAX_FEATURES = 5000 
EMBEDDING_DIMS = 64

train_filepath = "dataset/train.csv"
val_filepath = "dataset/val.csv"
test_filepath = "dataset/test.csv"

train_df = pd.read_csv(train_filepath, 
                index_col=0) 
val_df = pd.read_csv(val_filepath, 
                index_col=0) 
test_df = pd.read_csv(test_filepath, 
                index_col=0) 

In [17]:
train_df['Tweet'] = train_df['Tweet'].apply(lambda p: text_normalize(p)).astype(str) 
val_df['Tweet'] = val_df['Tweet'].apply(lambda p: text_normalize(p)).astype(str) 
test_df['Tweet'] = test_df['Tweet'].apply(lambda p: text_normalize(p)).astype(str) 

class_lst = np.array(train_df.columns[2:])
n_classes = len(class_lst)

X_train, y_train = train_df['Tweet'].to_numpy(), train_df[class_lst].astype('int').to_numpy()
X_val, y_val = val_df['Tweet'].to_numpy(), val_df[class_lst].astype('int').to_numpy()
X_test, y_test = test_df['Tweet'].to_numpy(), test_df[class_lst].astype('int').to_numpy()

In [18]:
train_df.head(3)

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2017-En-21441,worri payment problem may never joyc meyer m...,False,True,False,False,False,False,True,False,False,False,True
1,2017-En-31535,whatev decid make sure make happi,False,False,False,False,True,True,True,False,False,False,False
2,2017-En-21068,max_kellerman also help major nfl coach inept...,True,False,True,False,True,False,True,False,False,False,False


In [19]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

In [20]:
def inverse_label(class_lst, onehot_label):
    return class_lst[onehot_label > 0]

for text_batch, label_batch in train_ds.take(1):
  for i in range(10):
    print(f"Tweet {i}: ", text_batch.numpy()[i])
    print("Label:", inverse_label(class_lst, label_batch.numpy()[i]))

Tweet 0:  b'worri payment problem may never  joyc meyer  motiv leadership worri'
Label: ['anticipation' 'optimism' 'trust']
Tweet 1:  b'whatev decid make sure make happi'
Label: ['joy' 'love' 'optimism']
Tweet 2:  b'max_kellerman  also help major nfl coach inept bill obrien play call wow  gopat'
Label: ['anger' 'disgust' 'joy' 'optimism']
Tweet 3:  b'accept challeng liter even feel exhilar victori  georg patton'
Label: ['joy' 'optimism']
Tweet 4:  b'roommat okay cant spell autocorrect terribl firstworldprob'
Label: ['anger' 'disgust']
Tweet 5:  b'that cute atsu probabl shi photo cherri help uwu'
Label: ['joy']
Tweet 6:  b'think human sens recogn impend doom'
Label: ['anticipation' 'pessimism']
Tweet 7:  b'rooney fuck untouch isnt fuck dread depay look decentishtonight'
Label: ['anger' 'disgust']
Tweet 8:  b'pretti depress u hit pan ur favourit highlight'
Label: ['disgust' 'sadness']
Tweet 9:  b'bossupjae pussi weak heard stfu bitch  got threaten pregnant '
Label: ['anger' 'disgust']


In [21]:
# Cấu hình các tham số tối ưu cho việc đọc dữ liệu
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## **3. Xây dựng mô hình**

In [22]:
# Khai báo layer text vectorization
text_vectorization_layer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES, # Kích thước bộ từ vựng
    output_mode='int', # Giá trị token là chỉ mục của từ trong vocab
    output_sequence_length=MAX_SEQ_LEN # Số token tối đa trong 1 vector
)

train_text = train_ds.map(lambda text, labels: text) # Gọi `content` của toàn bộ mẫu dữ liệu trong tập train
text_vectorization_layer.adapt(train_text) # Xây dựng layer vectorization dựa trên dữ liệu tập train

In [23]:
# Xây dựng hàm khởi tạo model
def build_model(max_features, max_seq_len, embedding_dims, n_classes):
    model = tf.keras.Sequential([
        # Input layer (nhận vào 1 string)
        tf.keras.Input(shape=(1,), dtype='string', name='input_layer'), 

        # Text Vectorization Layer đã khai báo ở trên
        text_vectorization_layer, 

        # Embedding Layer (chuyển đổi các token thành các vector)
        tf.keras.layers.Embedding(input_dim=max_features+1, 
                                  output_dim=embedding_dims, 
                                  embeddings_initializer=tf.random_uniform_initializer(),
                                  mask_zero=True,
                                  name='embedding_layer'), 

        # Bi-LSTM Layer 1 
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, 
                                    return_sequences=True, 
                                    kernel_initializer=tf.initializers.GlorotUniform()),
                                    name='bilstm_layer_1'),
        # Bi-LSTM Layer 2
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, 
                                    return_sequences=True, 
                                    kernel_initializer=tf.initializers.GlorotUniform()),
                                    name='bilstm_layer_2'),
        # Bi-LSTM Layer 3
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, 
                                    return_sequences=False, 
                                    kernel_initializer=tf.initializers.GlorotUniform()),
                                    name='bilstm_layer_3'),
                                 
        # Dropout Layer 1
        tf.keras.layers.Dropout(0.2,
                                name='dropout_layer_1'),

        # Fully-connected Layer 1
        tf.keras.layers.Dense(64, 
                              activation='relu',
                              kernel_initializer=tf.initializers.GlorotUniform(),
                              name='fc_layer_1'),

        # Dropout Layer 2
        tf.keras.layers.Dropout(0.3,
                                name='dropout_layer_2'),

        # Fully-connected Layer 2
        tf.keras.layers.Dense(32, 
                              activation='relu',
                              kernel_initializer=tf.initializers.GlorotUniform(),
                              name='fc_layer_2'),

        # Dropout Layer 3
        tf.keras.layers.Dropout(0.3,
                                name='dropout_layer_3'),

        # Output Layer
        tf.keras.layers.Dense(n_classes, 
                              activation='sigmoid', 
                              kernel_initializer=tf.initializers.GlorotUniform(),
                              name='output_layer') 
    ],
    name='bilstm_model')

    return model

In [24]:
model = build_model(max_features=MAX_FEATURES, 
                    max_seq_len=MAX_SEQ_LEN,
                    embedding_dims=EMBEDDING_DIMS, 
                    n_classes=n_classes)
model.summary()

Model: "bilstm_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 128)              0         
 ectorization)                                                   
                                                                 
 embedding_layer (Embedding)  (None, 128, 64)          320064    
                                                                 
 bilstm_layer_1 (Bidirection  (None, 128, 128)         66048     
 al)                                                             
                                                                 
 bilstm_layer_2 (Bidirection  (None, 128, 128)         98816     
 al)                                                             
                                                                 
 bilstm_layer_3 (Bidirection  (None, 128)              98816     
 al)                                                  

## **4. Cấu hình mô hình**

In [ ]:
# Khai báo một số giá trị siêu tham số
EPOCHS = 30
LR = 1e-4

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LR), # Sử dụng optimizer Adam
    loss=tf.keras.losses.BinaryCrossentropy(), # Sử dụng hàm loss BinaryCrossEntropy
    metrics=['accuracy'] # Sử dụng thêm độ đo đánh giá Accuracy
)

## **5. Thực hiện huấn luyện**

In [ ]:
# Thực hiện huấn luyện
history = model.fit( 
    train_ds, # Huấn luyện với bộ train_ds
    validation_data=val_ds, # Đánh giá trên bộ val_ds
    epochs=EPOCHS # Huấn luyện với số lần lặp = số epochs
)

## **6. Đánh giá và trực quan hóa**

In [ ]:
# Đánh giá mô hình trên tập test
test_evaluation = model.evaluate(test_ds)

In [ ]:
# Đọc các kết quả huấn luyện mô hình qua từng epoch
train_loss, train_acc = history.history['loss'], history.history['accuracy'] # Đọc thông tin loss, acc trên tập train
val_loss, val_acc = history.history['val_loss'], history.history['val_accuracy'] # Đọc thông tin loss, acc trên tập val

plt.figure(figsize=(10, 10)) # Cài đặt kích thước khung ảnh

plt.subplot(2, 2, 1) # Khởi tạo khung ảnh cho training loss
plt.xlabel('Epochs') # Hiển thị tên trục hoành là 'Epochs'
plt.ylabel('Loss') # Hiển thị tên trục tung là 'Loss'
plt.title('Training loss') # Hiển thị title của khung ảnh hiện tại là 'Training Loss'
plt.plot(train_loss, color='green') # Vẽ đường giá trị loss trên tập train qua từng epoch (đường vẽ màu đỏ)

plt.subplot(2, 2, 2) # Khởi tạo khung ảnh cho training acc
plt.xlabel('Epochs') # Hiển thị tên trục hoành là 'Epochs'
plt.ylabel('Accuracy') # Hiển thị tên trục tung là 'Accuracy'
plt.title('Training accuracy') # Hiển thị title của khung ảnh hiện tại là 'Training accuracy'
plt.plot(train_acc, color='orange') # Vẽ đường giá trị accuracy trên tập train qua từng epoch (đường vẽ màu cam)

plt.subplot(2, 2, 3) # Khởi tạo khung ảnh cho val loss
plt.xlabel('Epochs') # Hiển thị tên trục hoành là 'Epochs'
plt.ylabel('Loss') # Hiển thị tên trục tung là 'Loss'
plt.title('Validation loss') # Hiển thị title của khung ảnh hiện tại là 'Validation loss'
plt.plot(val_loss, color='green') # Vẽ đường giá trị loss trên tập val qua từng epoch (đường vẽ màu đỏ)

plt.subplot(2, 2, 4) # Khởi tạo khung ảnh cho val acc
plt.xlabel('Epochs') # Hiển thị tên trục hoành là 'Epochs'
plt.ylabel('Accuracy') # Hiển thị tên trục tung là 'Accuracy'
plt.title('Validation accuracy') # Hiển thị title của khung ảnh hiện tại là 'Validation accuracy'
plt.plot(val_acc, color='orange') # Vẽ đường giá trị accuracy trên tập val qua từng epoch (đường vẽ màu cam)

plt.show() # Hiển thị 4 khung ảnh nhỏ

## **7. Inference**

In [ ]:
threshold = 0.4

for text_batch, label_batch in test_ds.take(1):
    for i in range(10):
        input_text = text_batch[i].numpy()
        label = label_batch[i].numpy()
        pred = model.predict(np.expand_dims(input_text, 0), verbose=0)[0]
        threshold_pred = np.where(pred > threshold, 1, 0)
        print(f"Text: {input_text}")
        print(f"Label: {inverse_label(class_lst, label)}")
        print(f"Predicted Label(s): ({inverse_label(class_lst, threshold_pred)})")
        print(" ")